In [70]:
import json
import pandas as pd
from pyodide.ffi import to_js
from IPython.display import JSON, HTML
from js import Object, fetch
from io import StringIO

In [ ]:
# Resultat als JSON

In [65]:
async def query_json(query_string, store = "L"):
    
    # the three Swiss triplestores
    if store == "F":
        address = 'https://fedlex.data.admin.ch/sparqlendpoint'
    elif store == "G":
        address = 'https://geo.ld.admin.ch/query'
    else:
        address = 'https://ld.admin.ch/query'
    
    try:
        resp = await fetch(address,
          method="POST",
          body="query=" + query_string,
          credentials="same-origin",
          headers=Object.fromEntries(to_js({"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", "Accept": "application/sparql-results+json" })),
        )
    except Exception:
        raise Abort()
    
    if resp.ok:
    
        res = await resp.text()

        try:
            data_json = json.loads(res)
        except:
            print(res)

        df = pd.json_normalize(data_json["results"]["bindings"], sep="_")

        return df
    else:
        print(resp.status)


In [66]:
df = await query_json("SELECT * WHERE {?s ?p ?o.} LIMIT 10", "L")
df = HTML(df.to_html(render_links=True, escape=False))
display(df)

,p_type,p_value,s_type,s_value,o_type,o_value
0,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8501607,uri,http://classifications.data.admin.ch/canton/
1,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8501950,uri,http://classifications.data.admin.ch/canton/
2,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8501611,uri,http://classifications.data.admin.ch/canton/
3,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8519449,uri,http://classifications.data.admin.ch/canton/
4,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8505592,uri,http://classifications.data.admin.ch/canton/
5,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8505581,uri,http://classifications.data.admin.ch/canton/
6,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8505594,uri,http://classifications.data.admin.ch/canton/
7,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8505578,uri,http://classifications.data.admin.ch/canton/
8,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8505584,uri,http://classifications.data.admin.ch/canton/
9,uri,https://gont.ch/canton,uri,https://lod.opentransportdata.swiss/didok/8518155,uri,http://classifications.data.admin.ch/canton/


# Resultat als CSV

In [73]:
async def query_csv(query_string, store = "L"):
    
    # the three Swiss triplestores
    if store == "F":
        address = 'https://fedlex.data.admin.ch/sparqlendpoint'
    elif store == "G":
        address = 'https://geo.ld.admin.ch/query'
    else:
        address = 'https://ld.admin.ch/query'
    
    try:
        resp = await fetch(address,
          method="POST",
          body="query=" + query_string,
          credentials="same-origin",
          headers=Object.fromEntries(to_js({"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", "Accept": "text/csv" })),
        )
    except Exception:
        raise Abort()
    
    if resp.ok:
    
        res = await resp.text()

        df = pd.read_csv(StringIO(res))

        return df
    else:
        print(resp.status)


In [74]:
df = await query_csv("SELECT * WHERE {?s ?p ?o.} LIMIT 10", "L")
df = HTML(df.to_html(render_links=True, escape=False))
display(df)

,s,p,o
0,https://lod.opentransportdata.swiss/didok/8501607,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
1,https://lod.opentransportdata.swiss/didok/8501950,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
2,https://lod.opentransportdata.swiss/didok/8501611,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
3,https://lod.opentransportdata.swiss/didok/8519449,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
4,https://lod.opentransportdata.swiss/didok/8505592,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
5,https://lod.opentransportdata.swiss/didok/8505581,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
6,https://lod.opentransportdata.swiss/didok/8505594,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
7,https://lod.opentransportdata.swiss/didok/8505578,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
8,https://lod.opentransportdata.swiss/didok/8505584,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
9,https://lod.opentransportdata.swiss/didok/8518155,https://gont.ch/canton,http://classifications.data.admin.ch/canton/
